In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np



In [3]:
# Prendre en main les données
data = pd.read_csv("../data/train.csv")
data

,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_221358.xml,"Feuilleté de saumon et de poireau, sauce aux c...",Plat principal,Facile,Moyen,- 1 gros pavé de saumon - 100 g de crevettes d...,Couper finement le blanc et un peu de vert des...
1,recette_48656.xml,Cake poulet/moutarde/amandes,Entrée,Très facile,Bon marché,- 3 œufs - 150 g de farine - 1 sachet de levur...,"Couper finement l'échalote, la faire revenir à..."
2,recette_30049.xml,Bûche à la truite fumée (7ème rencontre),Entrée,Moyennement difficile,Assez Cher,- 800 g de filet de truite saumonnée fumée en ...,Faites blanchir les épinards à l'eau bouillant...
3,recette_71424.xml,Gâteau au yaourt au coco sans huile de laetitia,Dessert,Très facile,Bon marché,- 1 pot de yaourt - 1 pot de lait de coco - 3 ...,Mélanger dans l'ordre tous les ingrédients en ...
4,recette_217204.xml,Crêpes au canard laqué,Entrée,Moyennement difficile,Moyen,- 90 g de farine - 45 g de maïzena - 2 œufs - ...,"Fouetter les œufs avec l'eau, le lait et le su..."
...,...,...,...,...,...,...,...
12468,recette_88545.xml,Tarte thon et courgettes,Plat principal,Facile,Bon marché,- 1 pâte feuilleté - 2 œufs - 2 courgettes moy...,"Couper les courgettes en fines rondelles, les ..."
12469,recette_36440.xml,Gratin de légumes à la viande,Plat principal,Facile,Bon marché,- 500 g de pommes de terre - 400 g de carotte...,Epluchez les pommes de terre et lavez-les. Cou...
12470,recette_174633.xml,Tagliatelles fraiches au Montepulciano,Plat principal,Difficile,Moyen,- Pour les tagliatelles : - 150 g de semoule f...,Je conseille de préparer les tagliatelles la v...
12471,recette_43030.xml,"Bavarois praliné pistache, coeur de mousse au ...",Dessert,Difficile,Assez Cher,- Pour le biscuit Dacquoise : - 160 g de blanc...,Petits commentaires :J'ai indiqué les recettes...


In [4]:
data.columns


Index(['doc_id', 'titre', 'type', 'difficulte', 'cout', 'ingredients',
       'recette'],
      dtype='object')

In [5]:
#methode Baseline

y = data['type']
data['combined'] = data['titre'] + data['recette']
X = data['combined']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

print(classification_report(y_test, y_pred))


                precision    recall  f1-score   support

       Dessert       0.31      0.32      0.31      1092
        Entrée       0.26      0.23      0.25       912
Plat principal       0.48      0.50      0.49      1738

      accuracy                           0.38      3742
     macro avg       0.35      0.35      0.35      3742
  weighted avg       0.38      0.38      0.38      3742



In [6]:
### Run2: TF-IDF
### Run3: Word2Vec
### Run4: 

train_reviews = X_train
test_reviews = X_test
train_reviews

8863     Boeuf mijoté aux 3 épicesDans une cocotte, fai...
393      SAUMON AU MAROILLES ET AU THYM, SAUCE MOUTARDE...
11399    Sauté de Dinde Caramélisée ( Miel/Citron)Coupe...
4139     Petits fours aux carottesEpluchez les carottes...
4192     Pie garnie au boeuf et aux rognonsNettoyez les...
                               ...                        
11964    Cake à l'orange, poisson et carottesPréchauffe...
5191     Bananes au citronCouper les bananes en rondell...
5390     Moussaka au thonLaver et trancher fin les aube...
860      Lasagnes à la ratatouille et au fromageEmincer...
7270     Cocotte de légumes d'hiver et chorizo douxNett...
Name: combined, Length: 8731, dtype: object

In [28]:
# ## SANS PREPROCESSING

# data = pd.read_csv("../data/train.csv")

# data['combined'] = data['titre'] + data['recette']

# X = data['combined']
# y = data['type']

# test = pd.read_csv("../data/test.csv")
# test['combined'] = test['titre'] + test['recette']
# X_testt = test['combined']
# y_testt = test['type']


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# tfidf = TfidfVectorizer()

# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_testt)


# svm = SVC(kernel='linear', C=1, random_state=42)
# svm.fit(X_train_tfidf, y_train)




# y_pred = svm.predict(X_test_tfidf)

# print(classification_report(y_test, y_pred))


## SANS PREPROCESSING

data = pd.read_csv("../data/train.csv")

data['combined'] = data['titre'] + data['recette']

X = data['combined']
y = data['type']

test = pd.read_csv("../data/test.csv")
test['combined'] = test['titre'] + test['recette']
X_testt = test['combined']
y_testt = test['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_testt)

svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_test_tfidf)

print(classification_report(y_testt, y_pred)) 

                precision    recall  f1-score   support

       Dessert       0.98      0.99      0.98       407
        Entrée       0.76      0.69      0.72       337
Plat principal       0.85      0.88      0.86       644

      accuracy                           0.87      1388
     macro avg       0.86      0.85      0.86      1388
  weighted avg       0.86      0.87      0.86      1388



In [8]:

stopwords = nltk.corpus.stopwords.words('french')

def clean_text(text):
    text = re.sub(r'<.*?>', '', text) #Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text) #Remove punctuation
    text = text.lower() #Convert to lowercase
    text = text.split() #Split the text
    text = [word for word in text if not word in set(stopwords)] #Remove stopwords
    text = ' '.join(text)
    return text

def stemmer(text):
    text = text.split()
    stemmer = nltk.PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = ' '.join(stemmed_words)
    return text

In [9]:

## AVEC PREPROCESSING
data = pd.read_csv("../data/train.csv")

data['combined'] = data['titre'] + data['recette']
data['combined'] = data['combined'].apply(clean_text)
data['combined'] = data['combined'].apply(stemmer)

X = data['combined']
y = data['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98      1092
        Entrée       0.77      0.64      0.70       912
Plat principal       0.83      0.89      0.86      1738

      accuracy                           0.86      3742
     macro avg       0.86      0.84      0.85      3742
  weighted avg       0.86      0.86      0.86      3742



In [10]:
 #Modèle Word2Vec

from sklearn.svm import SVC
corpus = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, sg=1)

# Calcul des vecteurs de mots moyens pour chaque document
def document_vector(word2vec_model, doc):
    """Calculer le vecteur moyen pour un document en utilisant les vecteurs de mots Word2Vec"""
    # filtrer les mots absents dans le vocabulaire
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) != 0:
        return np.mean(word2vec_model.wv[doc], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

X_train_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train])
X_test_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_test])

svm_classifier = SVC(random_state=42)
svm_classifier.fit(X_train_word2vec, y_train)

y_pred_word2vec = svm_classifier.predict(X_test_word2vec)

print("Classification Report Word2Vec:")
print(classification_report(y_test, y_pred_word2vec))


Classification Report Word2Vec:
                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98      1092
        Entrée       0.74      0.47      0.58       912
Plat principal       0.77      0.91      0.83      1738

      accuracy                           0.83      3742
     macro avg       0.83      0.79      0.80      3742
  weighted avg       0.82      0.83      0.81      3742



In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

vectorizer = CountVectorizer()

X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

svm_classifier = SVC(random_state=42)
svm_classifier.fit(X_train_counts, y_train)

y_pred_counts = svm_classifier.predict(X_test_counts)

# Print the classification report
print("Classification Report CountVectorizer:")
print(classification_report(y_test, y_pred_counts))

Classification Report CountVectorizer:
                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98      1092
        Entrée       0.79      0.61      0.69       912
Plat principal       0.82      0.91      0.86      1738

      accuracy                           0.86      3742
     macro avg       0.86      0.84      0.84      3742
  weighted avg       0.86      0.86      0.85      3742



In [12]:
#############    TEST  #############


In [23]:
test = pd.read_csv("../data/test.csv")
test['combined'] = test['titre'] + test['recette']

In [24]:
import pandas as pd

test = pd.read_csv("../data/test.csv")
test['combined'] = test['titre'] + test['recette']
X_test = test['combined']  # Assurez-vous d'ajuster 'text_column' en fonction du nom de la colonne contenant les données textuelles dans votre fichier CSV

# Transformer les données de test en vecteurs de comptage en utilisant le même vectorizer que celui utilisé pour les données d'entraînement
X_test_counts = vectorizer.transform(X_test)

# Prédire les étiquettes de classe pour les données de test en utilisant le modèle SVM entraîné
y_pred_counts = svm_classifier.predict(X_test_counts)

# Afficher le rapport de classification
print("Classification Report CountVectorizer:")
print(classification_report(y_test, y_pred_counts))


Classification Report CountVectorizer:


ValueError: Found input variables with inconsistent numbers of samples: [3742, 1388]